In [17]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [18]:
# set up duration vector
duration = [3,4,5,6,7]
thresh = [3,4,5]

date = '2023-01-20'

In [19]:
def run_model(duration, thresh, date):
    
    # create empty dataframe to append to
    df = pd.DataFrame()

    for d in duration:
        for t in thresh:

            ## load data
            load_thresh = 99
            path = '~/Documents/data/CBGdata/single_day_xgboost_ready/' + str(date) + '_duration_' + str(d) + '_threshold_' + str(load_thresh) + 'xx.csv'
            dt = pd.read_csv(path, index_col = False)

            # load correct model for duration and threshold
            model_xgb_2 = xgb.Booster()
            model_xgb_2.load_model('~/Documents/code/cbg_code/unipoc_xgb_models/duration_' + str(d) + '_threshold_' + str(t) + '_ratio_all_final_model.model')

            ids = dt['label']
            X = dt.drop(['label', 'id'], axis=1)
            
            # use the loaded XGBoost model
            model = model_xgb_2

            # run model on data
            y_pred_proba = model.predict(xgb.DMatrix(X))

            # load appropriate threshold from .csv file
            threshold = pd.read_csv('~/Documents/code/cbg_code/inpatient_CBG_xgb-main/unipoc_model_auroc_threshold/duration_' + str(d) + '_threshold_' + str(t) + '_ratio_all_auroc_threshold.csv', index_col = False)

            threshold_to_apply = threshold['threshold'][0]

            # return ids of patients with predicted probability greater than threshold
            ids_to_return = ids[y_pred_proba >= threshold_to_apply]

            # append to ids vector
            data = pd.DataFrame({'date': date, 'duration': d, 'threshold': t, 'ids': ids_to_return})
            df = pd.concat([df, data])

    return(df)

In [26]:
review_ids = run_model([3,4,5,6,7], [3,4,5], date)
print(review_ids)

          date  duration  threshold         ids
1   2023-01-20         3          3   204506026
4   2023-01-20         3          3   706436482
6   2023-01-20         3          3  1008055476
9   2023-01-20         3          3  1211480739
11  2023-01-20         3          3  1506603270
..         ...       ...        ...         ...
14  2023-01-20         7          4  1807453197
1   2023-01-20         7          5   311646549
3   2023-01-20         7          5   504346288
14  2023-01-20         7          5  1807453197
19  2023-01-20         7          5  2909506088

[142 rows x 4 columns]


In [ ]:
## load data
#path = '~/Documents/data/CBGdata/xgboost_ready/duration_7_threshold_3_single_day.csv'
load_thresh = 99
path = '~/Documents/data/CBGdata/single_day_xgboost_ready/' + str(date) + '_duration_' + str(dur) + '_threshold_' + str(load_thresh) + 'xx.csv'

dt = pd.read_csv(path, index_col = False)

In [ ]:
# load model
model_xgb_2 = xgb.Booster()
model_xgb_2.load_model('~/Documents/code/cbg_code/unipoc_xgb_models/duration_' + str(dur) + '_threshold_' + str(thresh) + '_ratio_all_final_model.model')

In [ ]:
dt

,max_by_day_1,max_by_day_2,max_by_day_3,min_by_day_1,min_by_day_2,min_by_day_3,median_by_day_1,median_by_day_2,median_by_day_3,day_N_1,...,prior_overall_min,prior_overall_max,prior_duration,prior_gradient,prior_sd_cbg,prior_mean_cbg,prior_cV,prior_N_tests,time_from_index,label
0,5.1,13.1,NaN,4.8,6.7,NaN,4.95,7.30,NaN,2,...,4.0,27.9,3128220.0,3.971127e-07,4.577919,12.792035,0.357873,113.0,1589820.0,103376224
1,19.6,16.2,NaN,14.6,9.3,NaN,17.60,12.75,NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107636034
2,19.1,17.9,NaN,4.9,6.4,NaN,10.70,8.00,NaN,5,...,4.2,27.4,231300.0,3.664714e-07,6.744672,13.883333,0.485811,12.0,8857560.0,703485385
3,16.4,17.9,NaN,11.2,11.3,NaN,14.70,15.05,NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,709446241


In [ ]:
ids = dt['label']
X = dt.drop(['label', 'id'], axis=1)
# y = dt['label'] # no labels in this prospective data

# use the loaded XGBoost model
model = model_xgb_2

In [ ]:
# run saved model
y_pred_proba = model.predict(xgb.DMatrix(X))

# y_true = y

In [ ]:
print(y_pred_proba)

[0.5721854  0.15147565 0.434264   0.10015137]


In [ ]:
ids

0    103376224
1    107636034
2    703485385
3    709446241
Name: label, dtype: int64

In [ ]:
# load appropriate threshold from .csv file
threshold = pd.read_csv('~/Documents/code/cbg_code/inpatient_CBG_xgb-main/unipoc_model_auroc_threshold/duration_' + str(dur) + '_threshold_' + str(thresh) + '_ratio_all_auroc_threshold.csv', index_col = False)

In [ ]:
threshold_to_apply = threshold['threshold'][0]
print(threshold)


      auroc  threshold
0  0.810533   0.467685


In [ ]:
# return ids of patients with predicted probability greater than threshold
ids_to_return = ids[y_pred_proba > threshold_to_apply]
print(ids_to_return)

0    103376224
Name: label, dtype: int64
